In [1]:
from IPython.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

# Índice <a id="ind"></a>

* [1 - Caminhos dos arquivos](#path)
* [2 - Bibliotecas](#bib)
* [3 - Preparação dos dados](#prep_dados)
* [4 - Treinamento dos  modelos](#trei_mod)
    - [4.1 - Random forest](#red_forest)
    - [4.2 - XG boost](#xgboost)
    - [4.3 - Random forrest com clusterização (k=2)](#rf_clus_2)
    - [4.4 - XGBoost com clusterização (k=2)](#xg_clas_2)    
    - [4.5 - Random forrest com clusterização (k=3)](#rf_clus_3)    
    - [4.6 - XGBoost com clusterização (k=3)](#xg_clas_3)    
    - [4.7 - Random forrest com clusterização (k=4)](#rf_clus_4)    
    - [4.8 - XGBoost com clusterização (k=4](#xg_clus_4)    
    - [4.9 - Random forest com PCA](#rf_pca)    
    - [4.10 - XGBoost com PCA](#xg_pca)


# 1 - Caminhos dos arquivos<a id="path"></a>
[Índice](#ind)

In [17]:
from pathlib import Path
import os
import sys

In [18]:
# adiciona modulos no python path
#sys.path.append('..//modulos')

# caminhos dos arquivos
current_path = os.getcwd()
path_dados=Path(current_path)/"dados"
path_modelos=Path(current_path)/"modelos"

#cria o diretório 'modelos'
if not os.path.isdir(path_modelos):
    os.makedirs(path_modelos) 

# 2 - Bibliotecas<a id="bib"></a>
[Índice](#ind)

In [19]:
# bibliotecas
import pandas as pd
pd.pandas.set_option('display.max_columns', None)

# importa modulos
import modulos.funcoes_ML as func_ml
import modulos.transformacoes_ML as trans_ml
from scipy import stats
import joblib

from sklearn.pipeline import Pipeline
from sklearn.model_selection import (
    GridSearchCV,
    train_test_split,
    RandomizedSearchCV
)

#modelos
import xgboost as xgb
from sklearn.ensemble import RandomForestRegressor

import importlib    #usar como: importlib.reload(nome_pack)

# 3 - Preparação dos dados<a id="prep_dados"></a>
[Índice](#ind)

In [20]:
#carrega os dados
df=pd.read_csv(path_dados/"CO2 Emissions_Canada.csv", low_memory=False)

#renomeia as variáveis
mapper={"Engine Size(L)":"Engine Size",
        "Fuel Consumption City (L/100 km)":"Fuel Consumption City",
        "Fuel Consumption Hwy (L/100 km)":"Fuel Consumption Hwy",
        "Fuel Consumption Comb (L/100 km)":"Fuel Consumption Comb A",
        "Fuel Consumption Comb (mpg)":"Fuel Consumption Comb B",
        "CO2 Emissions(g/km)":"CO2 Emissions",       
       }
df.rename(mapper=mapper, axis=1, inplace=True)

# Codifica variáveis categóricas
df_modelos = df.copy()
lab_enc = trans_ml.Mapeamento()
df_modelos = lab_enc.fit_transform(df_modelos)


In [21]:
# Divide os dados 
# -------------------------------------

# dados de teste e treinamento
df_train, df_test = train_test_split(df_modelos,test_size=0.2, random_state=1000)

# variável treinamento e alvo
col_alvo='CO2 Emissions'
col_treinamento=[var for var in df_modelos.columns if var != 'CO2 Emissions']

# dados de treinamento
X_train=df_train[col_treinamento].values
y_train=df_train[col_alvo].values

# dados de teste
X_test=df_test[col_treinamento].values
y_test=df_test[col_alvo].values


# 4 - Treinamento dos  modelos <a id="trei_mod"></a>

## 4.1- Random forest <a id="red_forest"></a>
[Índice](#ind)

In [470]:
# Cria o pipeline
# ===========================================
from sklearn.pipeline import Pipeline

# modelo de regressão
rand_forest= RandomForestRegressor(random_state=0,
                                   max_samples=None,
                                   bootstrap=True,)

# espaço de hiperparametros
param_grid = dict(
    max_depth=stats.randint(14, 18),
    max_features=stats.randint(2, 6),
    n_estimators=stats.randint(30, 50),
    )


# treina o modelo
search_pipe= Pipeline([    
    ('rand_search', RandomizedSearchCV(rand_forest,
                            param_grid,
                            scoring='neg_mean_squared_error',
                            cv=5,
                            n_iter = 100,        #numero de pontos no espaço de hiperp. testados
                            random_state=10000,  #semente pra gerador de num. alat.
                            n_jobs=-1,           #número de CPU's usados em paralelo
                            verbose=1,
                            refit=False))
    ])

# fita
search_pipe.fit(X_train, y_train)


Fitting 5 folds for each of 100 candidates, totalling 500 fits


Pipeline(steps=[('rand_search',
                 RandomizedSearchCV(cv=5,
                                    estimator=RandomForestRegressor(random_state=0),
                                    n_iter=100, n_jobs=-1,
                                    param_distributions={'max_depth': <scipy.stats._distn_infrastructure.rv_discrete_frozen object at 0x7f3217280e20>,
                                                         'max_features': <scipy.stats._distn_infrastructure.rv_discrete_frozen object at 0x7f3217281300>,
                                                         'n_estimators': <scipy.stats._distn_infrastructure.rv_discrete_frozen object at 0x7f3217269cf0>},
                                    random_state=10000, refit=False,
                                    scoring='neg_mean_squared_error',
                                    verbose=1))])

In [15]:
# -------- Como era sem o pipeline -------

#instancia do modelo
rand_forest= RandomForestRegressor(random_state=0,
                                   max_samples=None,
                                   bootstrap=True,)

# espaço de hiperparametros
param_grid = dict(
    max_depth=stats.randint(14, 18),
    max_features=stats.randint(2, 6),
    n_estimators=stats.randint(30, 50),
    )

#Define parâmetros da busca a busca
search = RandomizedSearchCV(rand_forest,
                            param_grid,
                            scoring='neg_mean_squared_error',
                            cv=5,
                            n_iter = 100,        #numero de pontos no espaço de hiperp. testados
                            random_state=10000,  #semente pra gerador de num. alat.
                            n_jobs=-1,           #número de CPU's usados em paralelo
                            verbose=1,
                            refit=False)
# fita
search.fit(X_train, y_train)


Fitting 5 folds for each of 100 candidates, totalling 500 fits


RandomizedSearchCV(cv=5, estimator=RandomForestRegressor(random_state=0),
                   n_iter=100, n_jobs=-1,
                   param_distributions={'max_depth': <scipy.stats._distn_infrastructure.rv_discrete_frozen object at 0x7f321e5872e0>,
                                        'max_features': <scipy.stats._distn_infrastructure.rv_discrete_frozen object at 0x7f322f3be9b0>,
                                        'n_estimators': <scipy.stats._distn_infrastructure.rv_discrete_frozen object at 0x7f321e712e90>},
                   random_state=10000, refit=False,
                   scoring='neg_mean_squared_error', verbose=1)

In [472]:
# salva o modelo
t=joblib.dump(search_pipe, path_modelos/"random_forest.joblib")

## 4.2 - XG boost <a id="xgboost">
[Índice](#ind)

In [473]:
%reset

Once deleted, variables cannot be recovered. Proceed (y/[n])? y


In [481]:
gbm_param_grid = {    
    'max_depth': range(2, 12),
    'n_estimators': range(10,50),
    'colsample_bytree': stats.uniform(0.3, 0.7),
    'learning_rate':stats.uniform(0.2, 0.6),    
    'max_leaves':[1,2,3],
    'max_bin': [255,256,257],
    'base_score': stats.uniform(0.2,0.7)
}

# Instancio o regressor gbm
gbm = xgb.XGBRegressor()

# treina o modelo
search_pipe= Pipeline([    
    ('rand_search', RandomizedSearchCV(gbm,
                            gbm_param_grid,
                            scoring='neg_mean_squared_error',
                            cv=5,
                            n_iter = 100,        #numero de pontos no espaço de hiperp. testados
                            random_state=1000,  #semente pra gerador de num. alat.
                            n_jobs=-1,           #número de CPU's usados em paralelo
                            verbose=1,
                            refit=False))
    ])

# fita
search_pipe.fit(X_train, y_train)


Fitting 5 folds for each of 100 candidates, totalling 500 fits


Pipeline(steps=[('rand_search',
                 RandomizedSearchCV(cv=5,
                                    estimator=XGBRegressor(base_score=None,
                                                           booster=None,
                                                           callbacks=None,
                                                           colsample_bylevel=None,
                                                           colsample_bynode=None,
                                                           colsample_bytree=None,
                                                           early_stopping_rounds=None,
                                                           enable_categorical=False,
                                                           eval_metric=None,
                                                           feature_types=None,
                                                           gamma=None,
                                                           gpu_id=None,
                                                           grow_policy=None,
                                                           importance_type=None,
                                                           interaction_...
                                                         'colsample_bytree': <scipy.stats._distn_infrastructure.rv_continuous_frozen object at 0x7f321e90be80>,
                                                         'learning_rate': <scipy.stats._distn_infrastructure.rv_continuous_frozen object at 0x7f32394d7e20>,
                                                         'max_bin': [255, 256,
                                                                     257],
                                                         'max_depth': range(2, 12),
                                                         'max_leaves': [1, 2,
                                                                        3],
                                                         'n_estimators': range(10, 50)},
                                    random_state=1000, refit=False,
                                    scoring='neg_mean_squared_error',
                                    verbose=1))])

In [482]:
# salva o modelo
t=joblib.dump(search_pipe, path_modelos/"XGBoost.joblib")

## 4.3 - Random forrest com clusterização (k=2)<a id="rf_clus_2"></a>
[Índice](#ind)

In [5]:
%reset

Once deleted, variables cannot be recovered. Proceed (y/[n])? y


In [14]:
# Cria o pipeline
# ========================================

#número de clusters
k=2

# modelo de regressão
rand_forest= RandomForestRegressor(random_state=0,
                                   max_samples=None,
                                   bootstrap=True,)

# espaço de hiperparametros
param_grid = dict(
    max_depth=stats.randint(14, 18),
    max_features=stats.randint(2, 6),
    n_estimators=stats.randint(30, 50),
    )


# treina o modelo
search_pipe= Pipeline([
    ('kmeans_cluster', trans_ml.AdicionaColClusters(k, rand=1000)),
    ('rand_search', RandomizedSearchCV(rand_forest,
                            param_grid,
                            scoring='neg_mean_squared_error',
                            cv=5,
                            n_iter = 100,        #numero de pontos no espaço de hiperp. testados
                            random_state=10000,  #semente pra gerador de num. alat.
                            n_jobs=-1,           #número de CPU's usados em paralelo
                            verbose=1,
                            refit=False))
    ])

# fita
search_pipe.fit(X_train, y_train)

Fitting 5 folds for each of 100 candidates, totalling 500 fits


Pipeline(steps=[('kmeans_cluster', AdicionaColClusters(k=2)),
                ('rand_search',
                 RandomizedSearchCV(cv=5,
                                    estimator=RandomForestRegressor(random_state=0),
                                    n_iter=100, n_jobs=-1,
                                    param_distributions={'max_depth': <scipy.stats._distn_infrastructure.rv_discrete_frozen object at 0x7fb821dcf310>,
                                                         'max_features': <scipy.stats._distn_infrastructure.rv_discrete_frozen object at 0x7fb83f6ca170>,
                                                         'n_estimators': <scipy.stats._distn_infrastructure.rv_discrete_frozen object at 0x7fb8215eb9d0>},
                                    random_state=10000, refit=False,
                                    scoring='neg_mean_squared_error',
                                    verbose=1))])

In [15]:
# Salva o modelo
# ========================
t=joblib.dump(search_pipe, path_modelos/"kmeans_2_randForest_.joblib")

## 4.4 - XGBoost com clusterização (k=2)<a id="xg_clas_2">
[Índice](#ind)

In [16]:
%reset

Once deleted, variables cannot be recovered. Proceed (y/[n])? y


In [22]:
# Cria o pipeline
# ===========================================

# número de clusters
k=2

# instancia do modelos
modelo = xgb.XGBRegressor()


# espaço de hiperparametros
param_grid = {    
    'max_depth': range(2, 12),
    'n_estimators': range(10,50),
    'colsample_bytree': stats.uniform(0.3, 0.7),
    'learning_rate':stats.uniform(0.2, 0.6),    
    'max_leaves':[1,2,3],
    'max_bin': [255,256,257],
    'base_score': stats.uniform(0.2,0.7)
}



# treina o modelo
search_pipe= Pipeline([
    ('kmeans_cluster', trans_ml.AdicionaColClusters(k,
                                                    rand=1000)),
    ('rand_search', RandomizedSearchCV(modelo,
                            param_grid,
                            scoring='neg_mean_squared_error',
                            cv=5,
                            n_iter = 100,        #numero de pontos no espaço de hiperp. testados
                            random_state=1000,   #semente pra gerador de num. alat.
                            n_jobs=-1,           #número de CPU's usados em paralelo
                            verbose=1,
                            refit=False))
    ])

# fita
search_pipe.fit(X_train, y_train)

Fitting 5 folds for each of 100 candidates, totalling 500 fits


Pipeline(steps=[('kmeans_cluster', AdicionaColClusters(k=2)),
                ('rand_search',
                 RandomizedSearchCV(cv=5,
                                    estimator=XGBRegressor(base_score=None,
                                                           booster=None,
                                                           callbacks=None,
                                                           colsample_bylevel=None,
                                                           colsample_bynode=None,
                                                           colsample_bytree=None,
                                                           early_stopping_rounds=None,
                                                           enable_categorical=False,
                                                           eval_metric=None,
                                                           feature_types=None,
                                                           gamma=None,
                                                           gpu_id=None,
                                                           grow_p...
                                                         'colsample_bytree': <scipy.stats._distn_infrastructure.rv_continuous_frozen object at 0x7fb8215e82e0>,
                                                         'learning_rate': <scipy.stats._distn_infrastructure.rv_continuous_frozen object at 0x7fb8218348e0>,
                                                         'max_bin': [255, 256,
                                                                     257],
                                                         'max_depth': range(2, 12),
                                                         'max_leaves': [1, 2,
                                                                        3],
                                                         'n_estimators': range(10, 50)},
                                    random_state=1000, refit=False,
                                    scoring='neg_mean_squared_error',
                                    verbose=1))])

In [23]:
# Salva o modelo
# ========================
t=joblib.dump(search_pipe, path_modelos/"kmeans_2_XGBoost_.joblib")

## 4.5 - Random forrest com clusterização (k=3)<a id="rf_clus_3"></a>
[Índice](#ind)

In [492]:
# Cria o pipeline
# ========================================

#número de clusters
k=3

# modelo de regressão
rand_forest= RandomForestRegressor(random_state=0,
                                   max_samples=None,
                                   bootstrap=True,)

# espaço de hiperparametros
param_grid = dict(
    max_depth=stats.randint(14, 18),
    max_features=stats.randint(2, 6),
    n_estimators=stats.randint(30, 50),
    )


# treina o modelo
search_pipe= Pipeline([
    ('kmeans_cluster', trans_ml.AdicionaColClusters(k, rand=1000)),
    ('rand_search', RandomizedSearchCV(rand_forest,
                            param_grid,
                            scoring='neg_mean_squared_error',
                            cv=5,
                            n_iter = 100,        #numero de pontos no espaço de hiperp. testados
                            random_state=10000,  #semente pra gerador de num. alat.
                            n_jobs=-1,           #número de CPU's usados em paralelo
                            verbose=1,
                            refit=False))
    ])

# fita
search_pipe.fit(X_train, y_train)


Fitting 5 folds for each of 100 candidates, totalling 500 fits


Pipeline(steps=[('kmeans_cluster', AdicionaColClusters()),
                ('rand_search',
                 RandomizedSearchCV(cv=5,
                                    estimator=RandomForestRegressor(random_state=0),
                                    n_iter=100, n_jobs=-1,
                                    param_distributions={'max_depth': <scipy.stats._distn_infrastructure.rv_discrete_frozen object at 0x7f32172ba5c0>,
                                                         'max_features': <scipy.stats._distn_infrastructure.rv_discrete_frozen object at 0x7f321e3d6920>,
                                                         'n_estimators': <scipy.stats._distn_infrastructure.rv_discrete_frozen object at 0x7f32172ba830>},
                                    random_state=10000, refit=False,
                                    scoring='neg_mean_squared_error',
                                    verbose=1))])

In [493]:
# Salva o modelo
# ========================
t=joblib.dump(search_pipe, path_modelos/"kmeans_3_randForest_.joblib")

## 4.6 - XGBoost com clusterização (k=3)<a id="xg_clas_3">
[Índice](#ind)

In [494]:
%reset

Once deleted, variables cannot be recovered. Proceed (y/[n])? y


In [500]:
# Cria o pipeline
# ===========================================

# número de clusters
k=3

# instancia do modelos
modelo = xgb.XGBRegressor()


# espaço de hiperparametros
param_grid = {    
    'max_depth': range(2, 12),
    'n_estimators': range(10,50),
    'colsample_bytree': stats.uniform(0.3, 0.7),
    'learning_rate':stats.uniform(0.2, 0.6),    
    'max_leaves':[1,2,3],
    'max_bin': [255,256,257],
    'base_score': stats.uniform(0.2,0.7)
}



# treina o modelo
search_pipe= Pipeline([
    ('kmeans_cluster', trans_ml.AdicionaColClusters(k,
                                                    rand=1000)),
    ('rand_search', RandomizedSearchCV(modelo,
                            param_grid,
                            scoring='neg_mean_squared_error',
                            cv=5,
                            n_iter = 100,        #numero de pontos no espaço de hiperp. testados
                            random_state=1000,   #semente pra gerador de num. alat.
                            n_jobs=-1,           #número de CPU's usados em paralelo
                            verbose=1,
                            refit=False))
    ])

# fita
search_pipe.fit(X_train, y_train)

Fitting 5 folds for each of 100 candidates, totalling 500 fits


Pipeline(steps=[('kmeans_cluster', AdicionaColClusters()),
                ('rand_search',
                 RandomizedSearchCV(cv=5,
                                    estimator=XGBRegressor(base_score=None,
                                                           booster=None,
                                                           callbacks=None,
                                                           colsample_bylevel=None,
                                                           colsample_bynode=None,
                                                           colsample_bytree=None,
                                                           early_stopping_rounds=None,
                                                           enable_categorical=False,
                                                           eval_metric=None,
                                                           feature_types=None,
                                                           gamma=None,
                                                           gpu_id=None,
                                                           grow_poli...
                                                         'colsample_bytree': <scipy.stats._distn_infrastructure.rv_continuous_frozen object at 0x7f32172bae00>,
                                                         'learning_rate': <scipy.stats._distn_infrastructure.rv_continuous_frozen object at 0x7f321e3d7fa0>,
                                                         'max_bin': [255, 256,
                                                                     257],
                                                         'max_depth': range(2, 12),
                                                         'max_leaves': [1, 2,
                                                                        3],
                                                         'n_estimators': range(10, 50)},
                                    random_state=1000, refit=False,
                                    scoring='neg_mean_squared_error',
                                    verbose=1))])

In [501]:
# Salva o modelo
# ========================
t=joblib.dump(search_pipe, path_modelos/"kmeans_3_XGBoost_.joblib")


## 4.7 - Random forrest com clusterização (k=4) <a id="rf_clus_4">
[Índice](#ind)

In [325]:
# Cria o pipeline
# ===========================================

# número de clusters
k=4

# modelo de regressão
rand_forest= RandomForestRegressor(random_state=0,
                                   max_samples=None,
                                   bootstrap=True,)

# espaço de hiperparametros
param_grid = dict(
    max_depth=stats.randint(14, 18),
    max_features=stats.randint(2, 6),
    n_estimators=stats.randint(30, 50),
    )


# treina o modelo
search_pipe= Pipeline([
    ('kmeans_cluster', trans_ml.AdicionaColClusters(k, rand=1000)),
    ('rand_search', RandomizedSearchCV(rand_forest,
                            param_grid,
                            scoring='neg_mean_squared_error',
                            cv=5,
                            n_iter = 100,        #numero de pontos no espaço de hiperp. testados
                            random_state=10000,  #semente pra gerador de num. alat.
                            n_jobs=-1,           #número de CPU's usados em paralelo
                            verbose=1,
                            refit=False))
    ])

# fita
search_pipe.fit(X_train, y_train)


Fitting 5 folds for each of 100 candidates, totalling 500 fits


Pipeline(steps=[('kmeans_cluster', AdicionaColClusters(k=4)),
                ('rand_search',
                 RandomizedSearchCV(cv=5,
                                    estimator=RandomForestRegressor(random_state=0),
                                    n_iter=100, n_jobs=-1,
                                    param_distributions={'max_depth': <scipy.stats._distn_infrastructure.rv_discrete_frozen object at 0x7f321750ee30>,
                                                         'max_features': <scipy.stats._distn_infrastructure.rv_discrete_frozen object at 0x7f321745acb0>,
                                                         'n_estimators': <scipy.stats._distn_infrastructure.rv_discrete_frozen object at 0x7f32175e6bf0>},
                                    random_state=10000, refit=False,
                                    scoring='neg_mean_squared_error',
                                    verbose=1))])

In [243]:
# Salva o modelo
# ========================
t=joblib.dump(search_pipe, path_modelos/"kmeans_4_randForest_.joblib")

## 4.8 - XGBoost com clusterização (k=4)<a id="xg_clus_4">
[Índice](#ind)

In [483]:
%reset

Once deleted, variables cannot be recovered. Proceed (y/[n])? y


In [488]:
# Cria o pipeline
# ===========================================

# número de clusters
k=4

# instancia do modelos
modelo = xgb.XGBRegressor()


# espaço de hiperparametros
param_grid = {    
    'max_depth': range(2, 12),
    'n_estimators': range(10,50),
    'colsample_bytree': stats.uniform(0.3, 0.7),
    'learning_rate':stats.uniform(0.2, 0.6),    
    'max_leaves':[1,2,3],
    'max_bin': [255,256,257],
    'base_score': stats.uniform(0.2,0.7)
}



# treina o modelo
search_pipe= Pipeline([
    ('kmeans_cluster', trans_ml.AdicionaColClusters(k,
                                                    rand=1000)),
    ('rand_search', RandomizedSearchCV(modelo,
                            param_grid,
                            scoring='neg_mean_squared_error',
                            cv=5,
                            n_iter = 100,        #numero de pontos no espaço de hiperp. testados
                            random_state=1000,   #semente pra gerador de num. alat.
                            n_jobs=-1,           #número de CPU's usados em paralelo
                            verbose=1,
                            refit=False))
    ])

# fita
search_pipe.fit(X_train, y_train)

Fitting 5 folds for each of 100 candidates, totalling 500 fits


Pipeline(steps=[('kmeans_cluster', AdicionaColClusters(k=4)),
                ('rand_search',
                 RandomizedSearchCV(cv=5,
                                    estimator=XGBRegressor(base_score=None,
                                                           booster=None,
                                                           callbacks=None,
                                                           colsample_bylevel=None,
                                                           colsample_bynode=None,
                                                           colsample_bytree=None,
                                                           early_stopping_rounds=None,
                                                           enable_categorical=False,
                                                           eval_metric=None,
                                                           feature_types=None,
                                                           gamma=None,
                                                           gpu_id=None,
                                                           grow_p...
                                                         'colsample_bytree': <scipy.stats._distn_infrastructure.rv_continuous_frozen object at 0x7f32174a5b70>,
                                                         'learning_rate': <scipy.stats._distn_infrastructure.rv_continuous_frozen object at 0x7f321e3d71c0>,
                                                         'max_bin': [255, 256,
                                                                     257],
                                                         'max_depth': range(2, 12),
                                                         'max_leaves': [1, 2,
                                                                        3],
                                                         'n_estimators': range(10, 50)},
                                    random_state=1000, refit=False,
                                    scoring='neg_mean_squared_error',
                                    verbose=1))])

In [490]:
# Salva o modelo
# ========================
t=joblib.dump(search_pipe, path_modelos/"kmeans_4_XGBoost_.joblib")


## 4.9 - Random forest com PCA<a id="rf_pca"></a>
[Índice](#ind)

## 4.10 - XGBoost com PCA<a id="xg_pca"></a>
[Índice](#ind)